In [1]:
import pandas as pd
import os
import math

In [2]:
fte = pd.read_csv(os.path.expanduser('~/Downloads/spi_matches.csv'))

In [ ]:
fte.columns

## Put two things online for every league
1. Table
2. Transformed aggregate version w/ one team like cte on sql query

In [3]:
def fte_last_season(path, league_dict):
    df = pd.read_csv(path)
    for key in league_dict:
        q = ''.join(['league_id == ', league_dict[key]])
        league = df.query(q).reset_index()
        num_teams = len(league.loc[:50, 'team1'].unique())
        i = math.floor(league.shape[0]/(num_teams * (num_teams - 1))) - 1
        return league.loc[((num_teams * (num_teams - 1)) * i):,:]

In [4]:
path = os.path.expanduser('~/Downloads/spi_matches.csv')
# league_dict = {'england':'2411'}
# league_dict = {'spain':'1869'}
#, 'italy':'1854',
league_dict = {'germany':'1845'} #, 'france':'1843'}
df = fte_last_season(path, league_dict)

In [5]:
df

,index,season,date,league_id,league,team1,team2,spi1,spi2,prob1,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
918,23396,2019,2019-08-16,1845,German Bundesliga,Bayern Munich,Hertha Berlin,93.98,66.45,0.8893,...,66.0,26.0,2.0,2.0,2.61,0.76,3.63,0.57,2.10,2.10
919,23425,2019,2019-08-17,1845,German Bundesliga,VfL Wolfsburg,FC Cologne,70.78,62.36,0.5561,...,31.0,27.2,2.0,1.0,0.58,0.96,1.16,0.86,2.10,1.05
920,23426,2019,2019-08-17,1845,German Bundesliga,Bayer Leverkusen,SC Paderborn,82.59,58.32,0.7970,...,60.2,37.4,3.0,2.0,2.27,1.27,1.15,0.87,3.15,2.10
921,23427,2019,2019-08-17,1845,German Bundesliga,SC Freiburg,Mainz,60.96,67.32,0.3941,...,33.4,22.1,3.0,0.0,1.83,0.96,1.23,1.45,2.38,0.00
922,23428,2019,2019-08-17,1845,German Bundesliga,Werder Bremen,Fortuna Düsseldorf,70.16,64.14,0.5235,...,27.0,25.2,1.0,3.0,1.84,0.96,2.20,1.06,1.05,3.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1219,31931,2019,2020-06-27,1845,German Bundesliga,FC Augsburg,RB Leipzig,61.65,85.45,0.2554,...,0.0,0.0,1.0,2.0,0.20,1.60,0.39,1.72,1.05,2.10
1220,31932,2019,2020-06-27,1845,German Bundesliga,Borussia Dortmund,TSG Hoffenheim,86.39,70.12,0.4950,...,0.0,0.0,0.0,4.0,0.19,3.54,1.23,2.66,0.00,4.20
1221,31933,2019,2020-06-27,1845,German Bundesliga,VfL Wolfsburg,Bayern Munich,75.62,93.72,0.2545,...,0.0,0.0,0.0,4.0,0.88,2.90,0.50,1.22,0.00,3.57
1222,31934,2019,2020-06-27,1845,German Bundesliga,Werder Bremen,FC Cologne,61.86,68.21,0.3829,...,64.8,0.0,6.0,1.0,3.16,0.89,1.53,0.51,6.30,1.05


In [24]:
fte.columns

Index(['index', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2'],
      dtype='object')

In [6]:
def fte_mvp(df):
    home = df.groupby('team1').apply(
        lambda x: (x['score1'] > x['score2']).sum() * 3 + (x['score1'] == x['score2']).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = ['score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1','adj_score2'], 
                        index = 'team1', aggfunc=sum)
    home = home.merge(pt.reset_index(), on = 'team1')
    pt = pd.pivot_table(df, values = 'date', index = 'team1', aggfunc=pd.Series.nunique)
    home = home.merge(pt.reset_index(), on = 'team1')
    cols = {'date':'played', 'team1':'team', 'score1':'scored', 'score2':'conceded', 'xg1':'xg_scored',
            'xg2':'xg_conceded', 'nsxg1':'nsxg_scored', 'nsxg2':'nsxg_conceded', 'adj_score1':'adj_goals_scored',
            'adj_score2':'adj_goals_conceded'}
    home = home.rename(columns = cols)
    away = df.groupby('team2').apply(
        lambda x: (x['score2'] > x['score1']).sum() * 3 + (x['score1'] == x['score2']).sum()).reset_index(name='points')
    pt = pd.pivot_table(df, values = ['score1', 'score2', 'xg1', 'xg2', 'nsxg1', 'nsxg2', 'adj_score1','adj_score2'], 
                        index = 'team2', aggfunc=sum)
    away = away.merge(pt.reset_index(), on = 'team2')
    pt = pd.pivot_table(df, values = 'date', index = 'team2', aggfunc=pd.Series.nunique)
    away = away.merge(pt.reset_index(), on = 'team2')
    cols = {'date':'played', 'team2':'team', 'score2':'scored', 'score1':'conceded', 'xg2':'xg_scored',
            'xg1':'xg_conceded', 'nsxg2':'nsxg_scored', 'nsxg1':'nsxg_conceded', 'adj_score2':'adj_goals_scored',
            'adj_score1':'adj_goals_conceded'}
    away = away.rename(columns = cols)
    t = pd.concat([home, away], ignore_index=True)
    t = t.groupby('team').sum()
    t['goal_difference'] = t['scored'] - t['conceded']
    return t

In [7]:
fte_mvp(df)

,points,adj_goals_scored,adj_goals_conceded,nsxg_scored,nsxg_conceded,scored,conceded,xg_scored,xg_conceded,played,goal_difference,xg_difference,nsxg_difference,adj_goal_difference
team,,,,,,,,,,,,,,
1. FC Union Berlin,41,41.52,58.90,35.36,50.73,41.0,58.0,44.22,52.74,34,-17.0,-8.52,-15.37,-17.38
Bayer Leverkusen,63,61.17,44.29,54.65,36.43,61.0,44.0,62.59,50.37,34,17.0,12.22,18.22,16.88
Bayern Munich,82,99.45,32.04,88.05,29.11,100.0,32.0,92.40,37.50,34,68.0,54.90,58.94,67.41
Borussia Dortmund,69,82.41,42.55,57.37,33.78,84.0,41.0,62.05,41.89,34,43.0,20.16,23.59,39.86
Borussia Monchengladbach,65,65.70,40.26,50.94,48.18,66.0,40.0,71.02,48.06,34,26.0,22.96,2.76,25.44
Eintracht Frankfurt,45,56.87,60.67,51.69,41.03,59.0,60.0,58.94,51.90,34,-1.0,7.04,10.66,-3.80
FC Augsburg,36,45.33,61.36,30.41,52.64,45.0,63.0,43.47,57.67,34,-18.0,-14.20,-22.23,-16.03
FC Cologne,36,50.77,68.47,43.47,48.01,51.0,69.0,49.51,53.88,34,-18.0,-4.37,-4.54,-17.70
Fortuna Düsseldorf,30,37.17,68.40,36.67,50.01,36.0,67.0,36.67,65.92,34,-31.0,-29.25,-13.34,-31.23


In [8]:
def fte_england_tb(df):
    t = fte_mvp(df)
    t = t.sort_values(by=['points', 'goal_difference', 'scored'], ascending = False)
    t['position'] = t.reset_index().index + 1
    return t

In [9]:
fte_england_tb(df)

,adj_goals_conceded,adj_goals_scored,conceded,nsxg_conceded,nsxg_scored,played,points,scored,xg_conceded,xg_scored,goal_difference,xg_difference,nsxg_difference,adj_goal_difference,position
team,,,,,,,,,,,,,,,
Liverpool,21.79,66.60,21.0,26.44,61.85,38,82,66.0,29.78,68.72,45.0,38.94,35.41,44.81,1
Manchester City,30.45,68.39,31.0,21.22,79.44,38,57,68.0,28.66,76.87,37.0,48.21,58.22,37.94,2
Leicester City,29.08,53.87,28.0,36.21,44.03,38,53,58.0,39.10,49.48,30.0,10.38,7.82,24.79,3
Chelsea,40.26,51.77,39.0,26.97,50.95,38,48,51.0,31.47,57.62,12.0,26.15,23.98,11.51,4
Manchester United,30.64,44.59,30.0,33.31,41.68,38,45,44.0,32.97,47.66,14.0,14.69,8.37,13.95,5
Wolverhampton,35.17,41.13,34.0,37.64,34.92,38,43,41.0,32.69,44.64,7.0,11.95,-2.72,5.96,6
Sheffield United,25.95,31.22,25.0,36.60,30.92,38,43,30.0,39.88,37.33,5.0,-2.55,-5.68,5.27,7
Tottenham Hotspur,41.58,48.32,40.0,46.10,36.02,38,41,47.0,46.26,45.70,7.0,-0.56,-10.08,6.74,8
Arsenal,37.04,40.97,36.0,41.37,41.27,38,40,40.0,43.29,39.50,4.0,-3.79,-0.10,3.93,9


In [24]:
def fte_spain_tb(df):
    t = fte_mvp(df)

    if t.duplicated(subset='points', keep = False).sum() > 0:
        t['tb'] = 0
        ties = t[t.duplicated(subset='points', keep = False)]['points'].unique()
        for tie in ties:
            teams = t[t['points'] == tie].index.values
            temp = df.loc[df['team1'].isin(teams) & df['team2'].isin(teams)].dropna()
            if len(temp.index) < (len(teams) * (len(teams) - 1)):
                for team in teams: t.at[team, 'tb'] = 1
            else:
                mini = fte_mvp(temp)
                if mini.duplicated(subset = ['points', 'goal_difference'], keep = False).sum() > 0:
                    mini = pd.merge(mini, t[['goal_difference']], how = 'inner', left_index=True, right_index=True, suffixes=['', '_total'])
                    mini = mini.sort_values(by=['points', 'goal_difference', 'goal_difference_total'], ascending = False)
                else:
                    mini = mini.sort_values(by=['points', 'goal_difference'], ascending = False)

                mini['tb'] = 2 * mini.shape[0] - mini.reset_index().index
                t.update(mini['tb'])
        t = t.sort_values(by=['points', 'tb', 'goal_difference', 'scored'], ascending = False)
        t.drop('tb', axis = 1, inplace=True)
    else:
        t = t.sort_values(by='points', ascending = False)
    t['position'] = t.reset_index().index + 1
    return t

In [23]:
t = fte_mvp(df)
teams = t[t['points'] == 33].index.values
df.loc[df['team1'].isin(teams) & df['team2'].isin(teams)]

,index,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
1191,25018,2019-09-24,1869,Spanish Primera Division,Real Betis,Levante,69.29,65.72,0.4991,0.2585,...,29.1,28.3,3.0,1.0,2.77,2.02,2.25,1.14,3.15,1.05
1456,32050,2020-04-22,1869,Spanish Primera Division,Levante,Real Betis,61.68,73.31,0.3484,0.4002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
fte_spain_tb(df)

,points,adj_goals_scored,adj_goals_conceded,nsxg_scored,nsxg_conceded,scored,conceded,xg_scored,xg_conceded,played,goal_difference,xg_difference,nsxg_difference,adj_goal_difference,position
team,,,,,,,,,,,,,,,
Barcelona,58,63.21,31.65,63.70,25.06,63.0,31.0,57.20,29.15,38,32.0,28.05,38.64,31.56,1
Real Madrid,56,47.85,19.74,58.70,24.86,49.0,19.0,53.74,26.73,38,30.0,27.01,33.84,28.11,2
Sevilla FC,47,39.54,30.26,39.48,24.26,39.0,29.0,41.35,29.03,38,10.0,12.32,15.22,9.28,3
Real Sociedad,46,45.68,33.36,37.07,35.55,45.0,33.0,43.81,33.65,38,12.0,10.16,1.52,12.32,4
Getafe,46,36.18,25.41,29.68,20.76,37.0,25.0,32.94,28.58,38,12.0,4.36,8.92,10.77,5
Atletico Madrid,45,31.48,21.54,36.07,25.94,31.0,21.0,44.48,24.81,38,10.0,19.67,10.13,9.94,6
Valencia,42,37.84,38.63,30.53,43.27,38.0,39.0,36.11,44.69,38,-1.0,-8.58,-12.74,-0.79,7
Villarreal,38,42.84,39.29,33.97,36.54,44.0,38.0,50.52,38.36,38,6.0,12.16,-2.57,3.55,8
Granada,38,33.80,31.08,22.46,29.39,33.0,32.0,30.83,33.86,38,1.0,-3.03,-6.93,2.72,9


In [8]:
def fte_germany_tb(df):
    # home
    t = fte_mvp(df)
    if t.duplicated(subset=['points', 'goal_difference', 'scored'], keep = False).sum() > 0:
        t['tb'] = 0
        ties = t[t.duplicated(subset='points', keep = False)]['points'].unique()
        for tie in ties:
            teams = t[t['points'] == tie].index.values
            temp = df.loc[df['team1'].isin(teams) & df['team2'].isin(teams)].dropna()
            mini = fte_mvp(temp)
            if mini.duplicated(subset = ['points', 'goal_difference'], keep = False).sum() > 0:
                mini_away = temp.loc[temp['team2'].isin(teams)].dropna()
                mini_away = mvp(mini_away)
                away = mvp(df,date = date, home_team = home_team, away_team = away_team, home_goals = home_goals, away_goals = away_goals)
                mini = pd.merge(mini, mini_away[['team','scored']], how = 'inner', left_index=True,
                                right_on = 'team', suffixes=['', '_away'])
                mini = pd.merge(mini, away[['team','scored']], how = 'inner', on = 'team_name',
                                suffixes=['', '_total_away'])
                mini = mini.sort_values(by=['points', 'goal_difference', 'scored_away', 'scored_total_away'], ascending = False)
            else:
                mini = mini.sort_values(by=['points', 'goal_difference'], ascending = False)
            mini['tb'] = 2 * mini.shape[0] - mini.reset_index().index
            t.update(mini['tb'])
        t = t.sort_values(by=['points', 'tb', 'goal_difference', 'scored'], ascending = False)
        t.drop('tb', axis = 1, inplace=True)
    else:
        t = t.sort_values(by=['points', 'goal_difference', 'scored'], ascending = False)
    t['position'] = t.reset_index().index + 1
    return t

In [9]:
alemania = fte_germany_tb(df)
alemania.to_csv()

,points,adj_goals_scored,adj_goals_conceded,nsxg_scored,nsxg_conceded,scored,conceded,xg_scored,xg_conceded,played,goal_difference,xg_difference,nsxg_difference,adj_goal_difference,position
team,,,,,,,,,,,,,,,
Bayern Munich,82,99.45,32.04,88.05,29.11,100.0,32.0,92.40,37.50,34,68.0,54.90,58.94,67.41,1
Borussia Dortmund,69,82.41,42.55,57.37,33.78,84.0,41.0,62.05,41.89,34,43.0,20.16,23.59,39.86,2
RB Leipzig,66,80.01,36.89,62.05,33.11,81.0,37.0,71.91,39.46,34,44.0,32.45,28.94,43.12,3
Borussia Monchengladbach,65,65.70,40.26,50.94,48.18,66.0,40.0,71.02,48.06,34,26.0,22.96,2.76,25.44,4
Bayer Leverkusen,63,61.17,44.29,54.65,36.43,61.0,44.0,62.59,50.37,34,17.0,12.22,18.22,16.88,5
TSG Hoffenheim,52,52.88,53.26,46.16,53.27,53.0,53.0,58.34,59.89,34,0.0,-1.55,-7.11,-0.38,6
VfL Wolfsburg,49,48.69,46.01,44.43,40.07,48.0,46.0,54.25,41.56,34,2.0,12.69,4.36,2.68,7
SC Freiburg,48,48.10,47.05,41.27,59.20,48.0,47.0,46.65,64.51,34,1.0,-17.86,-17.93,1.05,8
Eintracht Frankfurt,45,56.87,60.67,51.69,41.03,59.0,60.0,58.94,51.90,34,-1.0,7.04,10.66,-3.80,9
